In [1]:
import sys
#sys.path.append('/Users/pigard/CMS/ZZ_analysis')
#from plot_tools import *
import ROOT

Welcome to ROOTaaS 6.05/02


In [10]:
def make_ZpX_ZZMass(parametrization, binning = [100, 70, 1000]) :
    res = ROOT.TH1F('ZpX', '', binning[0], binning[1], binning[2])
    
    for param in parametrization :
        curr = ROOT.TH1F('ZpX_curr', '', binning[0], binning[1], binning[2])
        c_norm = param['norm']
        tf = ROOT.TF1("ZpX_curr", "landau(0)*(1 + exp( pol1(3))) + [5]*(TMath::Landau(x, [6], [7]))", 70, 1000)
        tf.SetParameters(4.404e-05,151.2,36.6,7.06,-0.00497,0.01446,157.3,26.00)
        plot_ratio = tf.Integral(binning[1], binning[2]) / tf.Integral(70, 1000)
        print 'plot_ratio ', plot_ratio
        curr.FillRandom('ZpX_curr', 100000)
        curr.Scale(c_norm * plot_ratio / curr.Integral())
        
        res.Add(curr)
        
    return res

def make_ZpX_OS(tree, proc_postfix, hist_name) :
    p_2P2F_SR = get_plot(prefix + tree + '_BKG_AllData' + proc_postfix + '.root', '2P2F/2P2F_' + hist_name)
    p_3P1F_SR = get_plot(prefix + tree + '_BKG_AllData' + proc_postfix + '.root', '3P1F/3P1F_' + hist_name)
    p_ZZ_3P1F_SR = get_plot(prefix + tree + '_BKG_ZZTo4l' + proc_postfix + '.root', '3P1F/3P1F_' + hist_name)
    
    p_3P1F_final = p_3P1F_SR.Clone()
    print '3P1F before ZZ removal: ', p_3P1F_final.Integral()
    p_3P1F_final.Add(p_ZZ_3P1F_SR, -1.)
    print '3P1F after ZZ removal: ', p_3P1F_final.Integral()
    p_3P1F_final.Add(p_2P2F_SR, -2.)
    print 'Final 3P1F : ', p_3P1F_final.Integral()

    p_BKG_total = p_2P2F_SR.Clone()
 
    def set_neg_bins_zero(histo ):
        for i in range(histo.GetNbinsX()) :
            if histo.GetBinContent(i) < 0. :
                histo.SetBinContent(i,0.)
    
    print 'Final 3P1F after negative bin removal: ', p_3P1F_final.Integral()
    #print 'FInal 2P2F above 300 GeV: ',p_2P2F_SR.Integral(, ) 
    p_BKG_total.Add(p_3P1F_final, 1.)
    
    return {'p_2P2F_SR' : p_2P2F_SR ,
            'p_3P1F_SR' : p_3P1F_SR,
            'p_3P1F_final' : p_3P1F_final,
            'p_ZZ_3P1F_SR': p_ZZ_3P1F_SR,
            'p_BKG_total' : p_BKG_total,
           }

In [6]:
prefix = '/Users/pigard/data/background_estimate/'
proc_postfix = '_12p9fb_own'
proc_postfix = '_30fb'
tree = 'REG'
selection = 'BLS'
directory = 'BLS'
region = '2P2F'
final_state = 'Zp2e'
zzjj_set = 'own'
vbf_plot_label = {'own' : 'MadGraph5 QCD=0 ZZ+2jets', 'official' : 'VBS, PHANTOM'}
ZZ_plot_label = {'own' : 'MadGraph5 QCD=4 ZZ+2jets', 'official' : 'qqZZ+0,1jets NLO'}
def plot(tree, hist_name) :
    plot_name = '%s/%s_%s'%(directory, selection, hist_name)
    data_plot = get_plot(prefix + tree + '_ZZ_ZZTo4l' + proc_postfix + '.root', plot_name) #_ZZ_AllData
    data_plot.Reset()
    zzjj_qcd = {}
    zzjj_qcd['official'] = get_plot(prefix + tree +'_ZZ_ZZTo4l' + proc_postfix + '.root', plot_name)
    zzjj_qcd['own'] = get_plot(prefix + tree +'_ZZ_ZZjj_qcd' + proc_postfix + '.root', plot_name)
    ZZ_plot = zzjj_qcd[zzjj_set]
    zzjj_ewk = {}
#     zzjj_ewk['official'] = sum_plots([ 
#                 [prefix + tree +'_ZZ_VBFTo2e2muJJ_0PMH125Contin_phantom128' + proc_postfix + '.root', plot_name],
#                 [prefix + tree +'_ZZ_VBFTo4muJJ_0PMH125Contin_phantom128' + proc_postfix + '.root', plot_name],
#                 [prefix + tree +'_ZZ_VBFTo4eJJ_0PMH125Contin_phantom128' + proc_postfix + '.root', plot_name],
#         ])
    zzjj_ewk['own'] = get_plot(prefix + tree +'_ZZ_ZZjj_ewk' + proc_postfix + '.root', plot_name)
    VBF_plot = zzjj_ewk[zzjj_set]
    #ZpX_plot = make_ZpX_OS(tree, proc_postfix, hist_name)['p_BKG_total']
    ttZ = get_plot(prefix + tree +'_ZZ_TTZ_MLM' + proc_postfix + '.root', plot_name)
    #WWZ = get_plot(prefix + tree +'_ZZ_WWZ' + proc_postfix + '.root', plot_name)
    irred = sum_plots([
                [prefix + tree +'_ZZ_TTZ_MLM' + proc_postfix + '.root', plot_name],
                #[prefix + tree +'_ZZ_WWZ' + proc_postfix + '.root', plot_name],
            ])
    #make_ZpX_ZZMass([{'norm' : 50}], [200, 0, 2000])
    ggZZ = sum_plots([ 
                [prefix + tree +'_ZZ_ggTo2e2mu_Contin_MCFM701' + proc_postfix + '.root', plot_name],
                [prefix + tree +'_ZZ_ggTo4e_Contin_MCFM701' + proc_postfix + '.root', plot_name],
                [prefix + tree +'_ZZ_ggTo4mu_Contin_MCFM701' + proc_postfix + '.root', plot_name],
                [prefix + tree +'_ZZ_ggTo2e2tau_Contin_MCFM701' + proc_postfix + '.root', plot_name],
                [prefix + tree +'_ZZ_ggTo2mu2tau_Contin_MCFM701' + proc_postfix + '.root', plot_name],
                [prefix + tree +'_ZZ_ggTo4tau_Contin_MCFM701' + proc_postfix + '.root', plot_name],        
                     ])
    save_plot([
            {'p' : data_plot, 
            'draw_options' : {'color' : ROOT.kBlack, 'legend_title' : 'Data (' + '{:.0f}'.format(data_plot.Integral()) + ' events)', 'MarkerStyle' : 20}},           
          ],
          stacked_plots = [
            {'p' : ttZ, 
            'draw_options' : {'fill_color' : ROOT.kOrange, 'line_color' : ROOT.kBlack,'legend_title' : 'ttZ (' + '{:.2f}'.format(ttZ.Integral()) + ' events)'}},                         
            #{'p' : WWZ, 
            #'draw_options' : {'fill_color' : ROOT.kOrange + 3, 'line_color' : ROOT.kBlack,'legend_title' : 'WWZ (' + '{:.2f}'.format(WWZ.Integral()) + ' events)'}},
            #{'p' : irred, 
            #'draw_options' : {'fill_color' : ROOT.kOrange, 'line_color' : ROOT.kBlack,'legend_title' : 'ttZ, WWZ (' + '{:.0f}'.format(irred.Integral()) + ' events)'}},            
            #{'p' : ZpX_plot, 
            #'draw_options' : {'fill_color' : 17, 'line_color' : ROOT.kBlack,'legend_title' : 'Z+X (' + '{:.2f}'.format(ZpX_plot.Integral()) + ' events)'}},            
            {'p' : ZZ_plot,
            'draw_options' : {'fill_color' : ROOT.kAzure - 4, 'line_color' : ROOT.kBlack,'legend_title' : ZZ_plot_label[zzjj_set] + ' (' + '{:.0f}'.format(ZZ_plot.Integral()) + ' events)'}},
            {'p' : ggZZ, 
            'draw_options' : {'fill_color' : ROOT.kAzure - 3, 'line_color' : ROOT.kBlack,'legend_title' : 'MCFM process 132 (' + '{:.2f}'.format(ggZZ.Integral()) + ' events)'}},                           
            {'p' : VBF_plot, 
            'draw_options' : {'fill_color' : 6 , 'line_color' : ROOT.kBlack,'legend_title' : vbf_plot_label[zzjj_set] + ' (' + '{:.2f}'.format(VBF_plot.Integral()) + ' events)'}},                           


            #            {'p' : get_plot(prefix + tree +'_CRZLL_DYJetsToLL_M50' + proc_postfix + '.root', region[0:4] + '/' + region  + '_ZZMass_' + final_state), 
            #'draw_options' : {'fill_color' : 30, 'line_color' : ROOT.kBlack,'legend_title' : 'DY+jets'}},

        ],
          draw_options = {'y_title' : 'Events', 'y_unit' : 'GeV',# 'y_log' : True,#'y_max' : 120,
                         # 'x_title' : 'm_{4l} [GeV]',
                         #'paves' : [{'text' :'L = 12.9 fb^{-1}', 'pos' : (0.6, 0.9, 0.9, 0.93)}],
                        'stack_draw_options' : 'LF2', 'y_min' : 0,
                          'legend_position' : (.5,.65,.89,.89),
                          'range' : (0.5, 1.0),
                          'y_max' : 8
                         }); #, 'draw_options' : 'HIST' # , 'y_log' : True , 'y_max' : 120

In [7]:
def ratio_plot(tree, hist_name) :
    canvas = ROOT.TCanvas( 'c1t', 'Test', 600, 700 )
    upper_pad = rt.TPad("upper_pad","upper_pad",0,0.35,1,1)
    upper_pad.SetBottomMargin(0)
    upper_pad.Draw()
    upper_pad.SetGridy()
    #upper_pad.cd()
    lower_pad = rt.TPad("lower_pad","lower_pad",0,0.05,1,0.35)
    lower_pad.SetTopMargin(0)
    lower_pad.SetBottomMargin(0.2)
    #lower_pad.Draw()
    lower_pad.cd()

    plot_name = '%s/%s_%s'%(directory, selection, hist_name)
    ttZ = get_plot(prefix + tree +'_ZZ_TTZ_MLM' + proc_postfix + '.root', plot_name)
    zzjj_ewk = get_plot(prefix + tree +'_ZZ_ZZjj_ewk' + proc_postfix + '.root', plot_name)
    
    ttZ.Divide(zzjj_ewk)
    save_plot([{'p' : ttZ, 
            'draw_options' : {'color' : ROOT.kBlack, 'legend_title' : 'Data (' + '{:.0f}'.format(ttZ.Integral()) + ' events)', 'MarkerStyle' : 20}},           
          ],
             # [{'p' : ttZ, 
            #'draw_options' : {'color' : ROOT.kBlack, 'legend_title' : 'Data (' + '{:.0f}'.format(ttZ.Integral()) + ' events)', 'MarkerStyle' : 20}},           
          #],
          draw_options = {'y_title' : 'Events', 'y_unit' : 'GeV',# 'y_log' : True,#'y_max' : 120,
                         # 'x_title' : 'm_{4l} [GeV]',
                         #'paves' : [{'text' :'L = 12.9 fb^{-1}', 'pos' : (0.6, 0.9, 0.9, 0.93)}],
                         'stack_draw_options' : 'LF2', 'y_min' : 0,
                          'legend_position' : (.5,.65,.89,.89),
                          'range' : (0.5, 1.0),
                          #'y_max' : 8
                          'canvas' : lower_pad
                         });
    

In [8]:
for p in [ 'sklearn_MVA_SR'] : #,'Njets_4p7_int', 'DiJetDEta_4p7','ZZMass_all', 'ZZjj_MVA','Njets_2p4', 'DiJetMass_4p7',  'jet_1_pt', 'jet_2_pt', 'jet_1_eta', 'jet_2_eta', 'Z_1_zepp', 'Z_2_zepp', 'rel_pt_hard',
         #'delta_rel','Z1Mass_all','Njets_4p7_cleaned'] :
    ratio_plot(tree, p)
    #plot(tree, p)

ReferenceError: attempt to access a null-pointer

In [61]:
from root_numpy import root2array, rec2array
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve

selection = 'ZZsel==120&&DiJetMass>100'
vbs_cut = '&&DiJetMass>400&&abs(DiJetDEta)>2.5'

background_set = []
signal_set = []
#ZZTo4l
#ZZjj_qcd
qqZZ = root2array('root://eosuser.cern.ch//eos//user/p/ppigard/cjlst_trees/160916/ZZTo4l/ZZ4lAnalysis.root', "ZZTree/candTree", ['ZZjj_MVA'],selection = selection)
qqZZ = rec2array(qqZZ[ ['ZZjj_MVA']])
qqZZ_cut = root2array('root://eosuser.cern.ch//eos//user/p/ppigard/cjlst_trees/160916/ZZTo4l/ZZ4lAnalysis.root', "ZZTree/candTree", ['ZZjj_MVA'],selection = selection+ vbs_cut)
qqZZ_cut = rec2array(qqZZ_cut[ ['ZZjj_MVA']])

#VBFTo2e2muJJ_0PMH125Contin_phantom128
VBF = root2array('root://eosuser.cern.ch//eos//user/p/ppigard/cjlst_trees/160916/ZZjj_ewk/ZZ4lAnalysis.root', "ZZTree/candTree", ['ZZjj_MVA'],selection = selection)
VBF = rec2array(VBF[ ['ZZjj_MVA']])
VBF_cut = root2array('root://eosuser.cern.ch//eos//user/p/ppigard/cjlst_trees/160916/ZZjj_ewk/ZZ4lAnalysis.root', "ZZTree/candTree", ['ZZjj_MVA'],selection = selection + vbs_cut)
VBF_cut = rec2array(VBF_cut[ ['ZZjj_MVA']])


VBF_eff = np.size(VBF_cut) / (1. * np.size(VBF))
qqZZ_eff = np.size(qqZZ_cut) / (1. * np.size(qqZZ))
print 'qqZZ_eff ', qqZZ_eff

X = np.concatenate((VBF, qqZZ))
y = np.concatenate((np.ones(VBF.shape[0]) ,
                        np.zeros(qqZZ.shape[0])))
fpr, tpr, thresholds = roc_curve(y, X, pos_label = 1.)

plt.plot(fpr, tpr, lw=1, label = 'CJLST')
#plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')
plt.plot(0.127, 0.657, lw=1,marker = 'x', label='VBS cuts', color = 'black')
plt.plot(qqZZ_eff, VBF_eff, lw=1,marker = 'x', label='VBS cuts on this dataset', color = 'red')

plt.xlim([-0.05, .4])
plt.ylim([0.4, 1.05])
plt.xlabel('background efficiency')
plt.ylabel('signal efficiency')
plt.legend(loc="best", numpoints = 1)
plt.grid()
plt.show()

IOError: unable to access tree 'ZZTree/candTree' in root://eosuser.cern.ch//eos//user/p/ppigard/cjlst_trees/160916/ZZTo4l/ZZ4lAnalysis.root

Error in <TMacOSXSystem::FindDynamicLibrary>: Netx[.so | .dll | .dylib | .sl | .dl | .a] does not exist in /Users/pigard/ROOT/install_dir_v6-05-02/lib:/Users/pigard/ROOT/install_dir_v6-05-02/lib::.:/Users/pigard/ROOT/install_dir_v6-05-02/lib::/usr/local/lib:/usr/X11R6/lib:/usr/lib:/lib:/lib/x86_64-linux-gnu:/usr/local/lib64:/usr/lib64:/lib64:
Error in <TMacOSXSystem::FindDynamicLibrary>: Netx[.so | .dll | .dylib | .sl | .dl | .a] does not exist in /Users/pigard/ROOT/install_dir_v6-05-02/lib:/Users/pigard/ROOT/install_dir_v6-05-02/lib::.:/Users/pigard/ROOT/install_dir_v6-05-02/lib::/usr/local/lib:/usr/X11R6/lib:/usr/lib:/lib:/lib/x86_64-linux-gnu:/usr/local/lib64:/usr/lib64:/lib64:


The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
from random import random
import ROOT as rt
rt.TH1.AddDirectory(rt.kFALSE)
import math

def norm_hist(hist, norm = 1.) :
    integral = hist.Integral()
    if integral > 0. : hist.Scale(norm / integral)
    return hist

def scale_hist(hist, scale) :
    hist.Scale(scale)
    return hist

def get_copy(filename, histname) :

    afile = ROOT.TFile(filename)

    #afile.ls()
    orgHist = afile.Get(histname)
    if orgHist == None :
        print 'Did not find histogram ', histname
        afile.ls()
    else :
        tmp = orgHist.Clone(orgHist.GetName() + '_copy' + str(random()) )
        #tmp.SetDirectory(False)
        afile.Close()
    return tmp

def get_plot(file_name, plot_name, options = None, reference_counts_name = None) :

    if options == None :
        options = {}
    out_plot = get_copy(file_name, plot_name)

    if 'Counters_hist' in options :
        counters = get_copy(file_name, 'Counters')
        sumOfWeights = counters.GetBinContent(40)
        factor = 1.
        if 'xsection' in options :
            xsection = options['xsection']
        return scale_hist(out_plot, xsection * sumOfWeights)

    if 'normalize' in options :
        return norm_hist(out_plot)

    if 'x_section' in options and 'reference_counts_name' is options :
        ref_counts_plot = get_copy(file_name, options['reference_counts_name'])
        return scale_hist(out_plot, options['x_section'] / ref_counts_plot.GetEntries())

    if 'x_section' in options :
        return norm_hist(out_plot, options['x_section'])

    if 'scale' in options :
        return scale_hist(out_plot, options['scale'])

    return out_plot


def sum_plots(plot_list = [], file_name = '', options = None) :
    if options == None :
        options = {}
        
    first_plot_item = plot_list[0]

    plots = []

    #if normalize :
    first_plot = get_plot(first_plot_item[0], first_plot_item[1])
    n_bins = first_plot.GetNbinsX()
    low_x = first_plot.GetXaxis().GetBinLowEdge(1)
    high_x = first_plot.GetXaxis().GetBinLowEdge(n_bins) + first_plot.GetXaxis().GetBinWidth(n_bins)

    out_plot = first_plot.Clone('sum_plots_' + str(random()))
    out_plot.Reset()

    for i_plot in range(0, len(plot_list)) :
        plot_item = plot_list[i_plot]
        plot = get_copy(plot_item[0], plot_item[1])

        if 'reference_counts_name' in options :
            ref_counts_plot = get_copy(plot_item[0], options['reference_counts_name'])
            get_plot(plot_item[0], plot_item[1])
            scale_hist(plot, x_section[i_plot] / ref_counts_plot.GetEntries())
        else :
            get_plot(plot_item[0], plot_item[1])

        plots.append(plot)

    for plot in plots:
        out_plot.Add(plot)

    if 'normalize' in options : norm_hist(out_plot)
    return out_plot
def to_precision(x,p):
    """
    returns a string representation of x formatted with a precision of p

    Based on the webkit javascript implementation taken from here:
    https://code.google.com/p/webkit-mirror/source/browse/JavaScriptCore/kjs/number_object.cpp
    """
    x = float(x)

    if x == 0.:
        return "0." + "0"*(p-1)

    out = []

    if x < 0:
        out.append("-")
        x = -x

    e = int(math.log10(x))
    tens = math.pow(10, e - p + 1)
    n = math.floor(x/tens)

    if n < math.pow(10, p - 1):
        e = e -1
        tens = math.pow(10, e - p+1)
        n = math.floor(x / tens)

    if abs((n + 1.) * tens - x) <= abs(n * tens -x):
        n = n + 1

    if n >= math.pow(10,p):
        n = n / 10.
        e = e + 1


    m = "%.*g" % (p, n)

    if e < -2 or e >= p:
        out.append(m[0])
        if p > 1:
            out.append(".")
            out.extend(m[1:p])
        out.append('e')
        if e > 0:
            out.append("+")
        out.append(str(e))
    elif e == (p -1):
        out.append(m)
    elif e >= 0:
        out.append(m[:e+1])
        if e+1 < len(m):
            out.append(".")
            out.extend(m[e+1:])
    else:
        out.append("0.")
        out.extend(["0"]*-(e+1))
        out.append(m)

    return "".join(out)

def get_y_title(histo, draw_options) :
    xAxis = histo.GetXaxis()

    unit_dict = {'': ['phi', '\phi', 'eta', '\eta'], 'GeV' : ['pT', 'pT', 'p_T', 'p_{T}', 'p_{t}', 'mass', 'm', 'm_{ZZ}', 'm_{4l}']  }

    if 'y_unit' not in draw_options :
        unit = None
        xTitle = xAxis.GetTitle()
        #print 'low: ', xTitle.find("[")+1, '  high ', xTitle.find("]")
        low = xTitle.find("[")
        if low > -1 :
            unit = xTitle[low + 1 : xTitle.find("]")]
        #print 'unit given ', unit ,  '  ', xTitle
        if unit == '' :
            unit = None
    else :
        unit = draw_options['y_unit']
    if unit is None :
        for k,v in unit_dict.iteritems() :
            for qty in v :
                if xTitle.find(qty) > -1 :
                    #print 'found ', qty, '   in ', xTitle
                    unit = k
                    continue
            if unit is not None :
                continue

    bin_width = xAxis.GetBinWidth(1)

    if isinstance(histo, ROOT.TGraphAsymmErrors) :
        x0 = ROOT.Double()
        x1 = ROOT.Double()
        t = ROOT.Double()
        histo.GetPoint(0, x0, t)
        histo.GetPoint(1, x1, t)
        bin_width = x1 - x0
    if 'y_title' in draw_options :
        y_title = draw_options['y_title']
    else:
        y_title = None
    if y_title is None :
        y_title = histo.GetYaxis().GetTitle()

    bin_width_string = to_precision(bin_width, 2)

    if bin_width_string != '0' and  'no_y_bin' not in draw_options :
        y_title = y_title + ' / ' + bin_width_string
    if unit is not None and  'no_y_bin' not in draw_options :
        y_title += ' ' + draw_options['y_unit']

        return y_title       

In [3]:
def save_plot(simple_plots = None, stacked_plots = None, draw_options = {}) :

    if simple_plots is None : simple_plots = []
    if stacked_plots is None : stacked_plots = []
    #  do_stacked_plots = False
    #  stacked_plots = []
    #else : do_stacked_plots = True

    _draw_options = draw_options.copy()

    if 'canvas' not in draw_options :
        canvas = ROOT.TCanvas( 'c1', 'Test', 200, 10, 700, 700 )
    else :
        canvas = draw_options['canvas']
    
    canvas = ROOT.TCanvas( 'c1', 'Test', 200, 10, 700, 700 )
    canvas.cd()
    #canvas.SetGridy()

    #print _draw_options
    canvas.SetLeftMargin(0.15)
    #canvas. SetLeftMargin(.5)
    histList = []
    if 'y_log' in _draw_options : canvas.SetLogy()

    if 'legend_position' not in _draw_options : _draw_options['legend_position'] = (.60,.70,.89,.89)

    if 'no_legend' not in _draw_options :
        legend = ROOT.TLegend(_draw_options['legend_position'][0], _draw_options['legend_position'][1], _draw_options['legend_position'][2], _draw_options['legend_position'][3])
        legend.SetBorderSize(0); # //no border for legend
        legend.SetFillColor(0);
        
    if len(stacked_plots) :
        first_plot = stacked_plots[0]['p']
    if len(simple_plots) :
        first_plot = simple_plots[0]['p']

    if not isinstance(first_plot, ROOT.TH1) :
        print 'is MultiGraph'
        theStack = ROOT.TMultiGraph()
    else :
        theStack = ROOT.THStack('tmp', 'tmp')
    if len(simple_plots) == 0 : theStack = None     

    if not isinstance(first_plot, ROOT.TH1) :
        theStackStack = ROOT.TMultiGraph()
    else :
        theStackStack = ROOT.THStack('tmp2', 'tmp')
    if len(stacked_plots) == 0 : theStackStack = None

    if 'x_title' not in _draw_options :
        _draw_options['x_title'] = first_plot.GetXaxis().GetTitle()

    if 'y_title' not in _draw_options or not 'no_y_bin' in _draw_options:
        _draw_options['y_title'] = get_y_title(first_plot, _draw_options) #yTitle = y_title, unit = _draw_options['y_unit'])

    if 'per_bin_norm' in _draw_options :
        plots = []
        for i in range(0, len(simple_plots)) :
            plots.append(simple_plots[i]['p'])

        norm_per_bin(plots)
        
    #print '_draw_options', _draw_options
    if 'draw_option' not in _draw_options :
        _draw_options['draw_option'] = 'NOSTACK'
    if 'stack_draw_option' not in _draw_options :
        _draw_options['stack_draw_option'] = 'HIST'

    for i in range(0, len(simple_plots)) :
        plot_bundle = simple_plots[i]
        h = plot_bundle['p']
        h.Draw()
        plot_draw_options = plot_bundle['draw_options']
        if 'color' in plot_draw_options :
            h.SetLineColor(plot_draw_options['color'])
            #h.SetFillColor(plot_draw_options['color'])
            h.SetMarkerColor(plot_draw_options['color'])
        h.SetLineWidth(2)
        if 'MarkerStyle' in plot_draw_options :
            h.SetMarkerStyle(plot_draw_options['MarkerStyle'])
        if 'no_legend' not in _draw_options :
            legend.AddEntry(h, plot_draw_options['legend_title'], "LP")
        theStack.Add(h)
        
    for i in range(0, len(stacked_plots)) :
        plot_bundle = stacked_plots[i]
        h = plot_bundle['p']
        h.Draw()
        plot_draw_options = plot_bundle['draw_options']
        if 'color' in plot_draw_options :
            h.SetLineColor(plot_draw_options['color'])
            h.SetFillColor(plot_draw_options['color'])
            h.SetMarkerColor(plot_draw_options['color'])
        if 'fill_color' in plot_draw_options :
            h.SetFillColor(plot_draw_options['fill_color'])
        if 'line_color' in plot_draw_options :
            h.SetLineColor(plot_draw_options['line_color'])

        h.SetLineWidth(2)
        if 'MarkerStyle' in plot_draw_options :
            h.SetMarkerStyle(plot_draw_options['MarkerStyle'])

        if 'no_legend' not in _draw_options :
            legend.AddEntry(h, plot_draw_options['legend_title'], "FP")
        theStackStack.Add(h)

    if theStack is not None :
        theStack.Draw("NOSTACK")
        theStack.GetXaxis().SetTitle(_draw_options['x_title'])
        theStack.GetYaxis().SetTitle(_draw_options['y_title'])
        theStack.GetYaxis().SetTitleOffset(1.7)

    if theStackStack is not None :
        theStackStack.Draw("STACK")
        if len(stacked_plots) > 0 :
            theStackStack.GetXaxis().SetTitle(_draw_options['x_title'])
            theStackStack.GetYaxis().SetTitle(_draw_options['y_title'])
            theStackStack.GetYaxis().SetTitleOffset(1.7)  
        
    if 'y_min' in _draw_options :
        if theStack is not None : theStack.SetMinimum(_draw_options['y_min'])
        if theStackStack is not None : theStackStack.SetMinimum(_draw_options['y_min'])

    if 'y_max' in _draw_options :
        if theStack is not None : theStack.SetMaximum(_draw_options['y_max'])
        if theStackStack is not None : theStackStack.SetMaximum(_draw_options['y_max'])
    else :
        if theStack is not None : theStack.Draw("NOSTACK")
        canvas.Update();
        stack_y_max = ROOT.gPad.GetUymax();
        
        if theStackStack is not None : theStackStack.Draw("")
        canvas.Update();
        stackstack_y_max = ROOT.gPad.GetUymax();

        y_max = max(stack_y_max, stackstack_y_max)
        # print 'y_max ', y_max
        if theStackStack is not None : theStackStack.SetMaximum(y_max)
        if theStack is not None : theStack.SetMaximum(y_max)

    if 'range' in draw_options :
        if theStackStack is not None : theStackStack.GetXaxis().SetRangeUser(draw_options['range'][0], draw_options['range'][1])
        if theStack is not None : theStack.GetXaxis().SetRangeUser(draw_options['range'][0], draw_options['range'][1])

    if theStackStack is not None :
        theStackStack.Draw(_draw_options['stack_draw_option'])
    
    if theStack is not None :
        same_string = 'SAME'
        if theStackStack is None : same_string = ''
        theStack.Draw(_draw_options['draw_option'] + same_string)
    else :
        theStack.Draw(_draw_options['draw_option'])

    if 'no_legend' not in _draw_options :
        legend.Draw("SAME")
        
    if 'paves' in _draw_options :
        for pave in _draw_options['paves'] :
            #print 'Preparing pave', pave['text']
            aPave = ROOT.TPaveText(pave['pos'][0],pave['pos'][1], pave['pos'][2], pave['pos'][3], "blNDC")
            aPave.SetFillStyle(0)
            aPave.SetBorderSize(0)
            aPave.Draw()
            aPave.AddText(pave['text'])
            aPave.Draw()
    canvas.Update()

    save_name = 'blubb'
    if 'save_name' in _draw_options :
        save_name = _draw_options['_draw_options']
        save_folder = save_name[:save_name.rfind('/')]
        save_file_name = save_name[save_name.rfind('/'):]

        subfolderName = outputFolderName + '//' + 'new_save/' + save_folder

        if not os.path.exists(subfolderName):
            os.makedirs(subfolderName)

        for fileFormat in savedFileFormats :
            canvas.Print(subfolderName + '//' + save_file_name + fileFormat)

    #theStack.Draw("NOSTACK"), _draw_options
    if 'do_not_draw_canvas' not in _draw_options :
        canvas.Draw()
    return theStack, legend      